# Building and flashing firmware

In order to flash the chip with the right firmware you need to edit the makefile with the good firmware path


## Connect the Firmware 

Build the firmware with this command :

In [ ]:
%%bash
cd ../hardware/victims/firmware/simpleserial-base/
make PLATFORM='CW308_STM32F3' CRYPTO_TARGET=NONE

Then, upload your firmware on the target :

In [ ]:
#cw.program_target(scope, cw.programmers.STM32FProgrammer, "chemin/de/votre/firmware.hex")

*For other targets, you'll need to use an external programmer or a debugger to flash the firmware onto the target.

## Communication with the Target

Communication with targets, which is done through the `SimpleSerial target` object we got earlier, is grouped into two categories:

1. Raw serial via `target.read()`, `target.write()`, `target.flush()`, etc. 

1. SimpleSerial commands via `target.simpleserial_read()`, `target.simpleserial_write()`, `target.simpleserial_wait_ack()`, etc.

The firmware we uploaded uses the simpleserial protocol (https://wiki.newae.com/SimpleSerial), so we'll start off with simpleserial. Later, we'll use the raw serial commands to send the same messages.

If you check the simpleserial-base firmware (`simpleserial-base.c`) you'll find that for the simpleserial `'p'` command, the target will echo back the command.

Now test if your hadware respond to your commands

## Capturing traces 

For capture trace let's arm the target : 

1. Arm the ChipWhisperer with `scope.arm()`. It will begin capturing as soon as it is triggered (which in our case is a rising edge on `gpio4`.
1. `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return will tell you whether or not the scope timed out. It does not return the captured scope data.
1. You can read back the captured power trace with `scope.get_last_trace()`.

`simpleserial_base` will trigger the ChipWhisperer when we send the `'p'` command. Try capturing a trace now:

ChipWhisperer also has a `capture_trace()` convience function that:

1. Optionally sends the `'k'` command
1. Arms the scope
1. Sends the `'p'` command
1. Captures the trace
1. Reads the return `'r'` message
1. Returns a `Trace` class that groups the trace data, `'p'` message, the `'r'` message, and the `'k'` message

It isn't always the best option to use, but it's usually sufficient for most simpleserial applications

## Troubleshooting
If you get the following error :

```console
.
Compiling:
    simpleserial-homemade-rsa.c ...In file included from utils/utils.h:4,
                 from simpleserial-homemade-rsa.c:3:
/usr/lib/gcc/arm-none-eabi/13.2.0/include/stdint.h:9:16: fatal error: stdint.h: No such file or directory
    9 | # include_next <stdint.h>
      |                ^~~~~~~~~~
compilation terminated.
```


You need to install the following package: `arm-none-eabi-newlib`